In [ ]:
import re
import csv
import gensim
from gensim import models,corpora
import pandas as pd
import snowballstemmer
from nepalitokenizer import NepaliTokenizer
import glob
import os
from os import path
import base64
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
from scipy.stats import entropy
from scipy.spatial import distance

In [ ]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../backend')

from utils import *

In [ ]:
df = pd.read_csv("../data/news-setopati/news_setopati_preprocessed_1.csv")

In [ ]:
'''
Reading saved dictionary and bow_corpus 
'''
id2word = corpora.Dictionary.load('../saved_model/dictionary_2')

# reading bow_corpus
file_path = '../saved_model/bow_corpus_2.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()
bow_corpus = pd.Series(lines, name='body')

In [ ]:
df

In [ ]:

#lda_model = models.ldamodel.LdaModel.load('../saved_model/lda_model_politics_2')

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                       id2word=id2word,
                                       num_topics=30, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=100,
                                       per_word_topics=True,
                                       workers=8)
# lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=40, id2word=id2word, passes=50, workers=5)

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=100,
                                           alpha=a,
                                           eta=b,
                                           workers=8)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()